In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
import sympy as sym

import models
import selection_functions
import symbolics

In [2]:
%matplotlib inline
plt.style.use("ggplot")

# Monomorphic $\gamma$ model

## Only G allele present 

Set $x_2=1-x_1$, $x_3=x4=0$ so that only the G allele of the $\gamma$ gene is present

In [3]:
x1, x2, x3 = sym.symbols("x1, x2, x3", real=True, nonnegative=True)
x4 = 1 - x1 - x2 - x3
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)
M, m = sym.symbols("M, m", real=True, nonnegative=True)
epsilon = sym.symbols("epsilon", real=True, positive=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

In [4]:
x = np.array([[x1], [x2], [x3], [x4]])
payoff_kernel = np.array([[R, S], [T, P]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, M, m, epsilon)

In [5]:
male_genotype_fitness = W.sum(axis=0).sum(axis=0)

In [6]:
male_GA_fitness = sym.together(sym.expand(male_genotype_fitness[0]))

In [7]:
male_GA_fitness

4.0*x1*(-M*UGA(x1 + x3) + R*UGA(x1 + x3)**2 + R*UgA(x1 + x3)**2 - S*UGA(x1 + x3)**2 + S*UGA(x1 + x3) - S*UgA(x1 + x3)**2 + S*UgA(x1 + x3) - m*UgA(x1 + x3))/(x1 + x3)

Fitness of males carrying the $GA$ genotype.

$$ 4\big((R - S)U_{GA}(x_1)^2 + (S - M)U_{GA}(x_1)\big) + 4\big((R - S)U_{gA}(x_1)^2 + (S - m)U_{gA}(x_1)\big) $$

In [8]:
male_Ga_fitness = sym.together(sym.expand(male_genotype_fitness[1]))

In [9]:
male_Ga_fitness

x2*(4.0*M*UGA(x1 + x3) - 4.0*M + 4.0*P*UGA(x1 + x3)**2 - 8.0*P*UGA(x1 + x3) + 4.0*P*UgA(x1 + x3)**2 - 8.0*P*UgA(x1 + x3) + 8.0*P - 4.0*T*UGA(x1 + x3)**2 + 4.0*T*UGA(x1 + x3) - 4.0*T*UgA(x1 + x3)**2 + 4.0*T*UgA(x1 + x3) + 4.0*m*UgA(x1 + x3) - 4.0*m)/(-x1 - x3 + 1)

Fitness of males carrying the $Ga$ genotype.

$$ 4\big(2P - (M + m)\big) + 4\big((T + M - 2P)U_{GA}(x_1) - (T - P)U_{GA}(x_1)^2\big) + 4\big((T + m - 2P)U_{gA}(x_1) - (T - P)U_{gA}(x_1)^2\big) $$

In [10]:
sym.collect(male_GA_fitness - male_Ga_fitness, (UGA(x1), UgA(x1)))

4.0*x1*(-M*UGA(x1 + x3) + R*UGA(x1 + x3)**2 + R*UgA(x1 + x3)**2 - S*UGA(x1 + x3)**2 + S*UGA(x1 + x3) - S*UgA(x1 + x3)**2 + S*UgA(x1 + x3) - m*UgA(x1 + x3))/(x1 + x3) - x2*(4.0*M*UGA(x1 + x3) - 4.0*M + 4.0*P*UGA(x1 + x3)**2 - 8.0*P*UGA(x1 + x3) + 4.0*P*UgA(x1 + x3)**2 - 8.0*P*UgA(x1 + x3) + 8.0*P - 4.0*T*UGA(x1 + x3)**2 + 4.0*T*UGA(x1 + x3) - 4.0*T*UgA(x1 + x3)**2 + 4.0*T*UgA(x1 + x3) + 4.0*m*UgA(x1 + x3) - 4.0*m)/(-x1 - x3 + 1)

Fitness differential is

$$ 4\big((T + R - P - S)U_{GA}(x_1)^2 - (T + 2M - 2P - S)U_{GA}(x_1)\big) + 4\big((T + R - P - S)U_{gA}(x_1)^2 - (T + 2m - 2P - S)U_{gA}(x_1)\big) - 4\big(2P - (M + m)\big) $$

In [11]:
def plot_male_genotype_fitness(x1, x2, x3, selection_function, d1, d3,
                               T, R, P, S, M, m, epsilon, max_time):

    fig, axes = plt.subplots(1,2, figsize=(15,8), sharex=True)

    # prepare the axes
    axes[0].set_ylim((0, 1.05))
    axes[0].set_xlabel(r"Time, $t$", fontsize=15)
    axes[0].set_ylabel(r"Offspring genotype shares, $x_i$", fontsize=15)

    axes[1].set_xlabel(r"Time, $t$", fontsize=15)
    axes[1].set_ylabel(r"Male genotype fitness", fontsize=15)

    # create the initial condition
    x4 = 1 - x1 - x2 - x3
    y0 = np.array([x1, x2, x3, x4])
    assert y0.sum() <= 1

    # create the payoff kernel
    assert (T > R) and (R > P) and (R > S), "Payoffs must satisfy either Prisoner's Dilemma or Stag Hunt constraints! T={}, R={}, P={}, S={}".format(T, R, P, S)
    payoff_kernel = np.array([[R, S], [T, P]])

    # create the selection functions
    if selection_function == "kirkpatrick":
        UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d1)
        UgA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d3)
    elif selection_function == "seger":
        UGA = lambda x_A: selection_functions.seger_selection(x_A, d1)
        UgA = lambda x_A: selection_functions.seger_selection(x_A, d3)
    elif selection_function == "wright":
        UGA = lambda x_A: selection_functions.wright_selection(x_A, d1)
        UgA = lambda x_A: selection_functions.wright_selection(x_A, d3)
    else:
        valid_funcs = ("kirkpatrick", "seger", "wright")
        msg = "Selection_function must be one of {}, {}, or {}.".format(*valid_funcs)
        raise ValueError(msg)

    # simulate the model starting from a random initial condition
    def f(t, y):
        W = models.generalized_sexual_selection(y, UGA, UgA, payoff_kernel, M, m, epsilon)
        y_dot = models.offspring_genotypes_evolution(W, y)
        return y_dot

    solution = integrate.solve_ivp(f, t_span=(0, max_time), y0=y0, method="RK45",
                                   rtol=1e-9, atol=1e-12, dense_output=True, vectorized=True)

    axes[0].plot(solution.t, solution.y[0], label="GA")
    axes[0].plot(solution.t, solution.y[1], label="Ga")
    axes[0].plot(solution.t, solution.y[2], label="gA")
    axes[0].plot(solution.t, solution.y[3], label="ga")
    axes[0].legend()

    def _male_genotype_fitness(yt):
        W = models.generalized_sexual_selection(yt, UGA, UgA, payoff_kernel, M, m, epsilon)
        male_genotype_fitness = W.sum(axis=0).sum(axis=0).reshape(-1, 1)
        return male_genotype_fitness

    def male_genotype_fitness(y):
        _, T = y.shape
        ws = []
        for t in range(T):
            yt = y[:,[t]]
            w = _male_genotype_fitness(yt)
            ws.append(w)
        return np.hstack(ws)

    fitness = male_genotype_fitness(solution.y)
    axes[1].plot(solution.t, fitness[0], label="GA")
    axes[1].plot(solution.t, fitness[1], label="Ga")
    axes[1].plot(solution.t, fitness[2], label="gA")
    axes[1].plot(solution.t, fitness[3], label="ga")
    axes[1].legend()

    plt.show()

    return (solution, fitness)

In [12]:
# sliders used to control the initial condition
x1_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=1e-3, description=r"$x_1$", readout_format=".3f")
x2_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=1e-3, description=r"$x_2$", readout_format=".3f")
x3_slider = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=1e-3, description=r"$x_3$", readout_format=".3f")

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# sliders used to control the metabolic costs
M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M_G$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m_g$")

# slider used to control which selection function is being used
U_slider = widgets.Dropdown(options=["kirkpatrick", "seger", "wright"], index=0, description=r"$U_{\gamma(j)A}$")

# slider that controls the parameters of the selection function
d1_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_1$")
d3_slider = widgets.FloatSlider(value=1, min=0.0, max=10, step=0.05, description=r"$d_3$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

# slider that controls max simulation time
max_time_slider = widgets.IntSlider(value=25, min=1, max=100000, description=r"$\max t$")

w = widgets.interactive(plot_male_genotype_fitness, x1=x1_slider, x2=x2_slider, x3=x3_slider,
                        selection_function=U_slider, d1=d1_slider, d3=d3_slider, 
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider,
                        M=M_slider, m=m_slider, epsilon=e_slider,
                        max_time=max_time_slider)
display(w)

A Jupyter Widget